## Viewsets
In python, a viewset is used in Django REST Framework to combine the logic for a set of related views into a single class. This allows for more efficient and organized code when dealing with CRUD (Create, Read, Update, Delete) operations on a model. There are many types of viewsets:
1. **ModelViewSet**: This is a complete set of views that provides default implementations for standard actions like list, create, retrieve, update, partial_update, and destroy. It is typically used when you want to expose all CRUD operations for a model.

   Example:
   ```python
   from rest_framework import viewsets
   from .models import MyModel
   from .serializers import MyModelSerializer

   class MyModelViewSet(viewsets.ModelViewSet):
       queryset = MyModel.objects.all()
       serializer_class = MyModelSerializer
   ```
2. **ReadOnlyModelViewSet**: This viewset provides only read-only operations, specifically list and retrieve actions. It is useful when you want to expose data without allowing modifications. 

   Example:
   ```python
   from rest_framework import viewsets
   from .models import MyModel
   from .serializers import MyModelSerializer

   class MyModelReadOnlyViewSet(viewsets.ReadOnlyModelViewSet):
       queryset = MyModel.objects.all()
       serializer_class = MyModelSerializer
   ```
3. **GenericViewSet**: This is a more flexible viewset that allows you to combine mixins to create custom behavior. You can choose which actions to include by mixing in the appropriate classes.
    Example:
    ```python
    from rest_framework import viewsets, mixins
    from .models import MyModel
    from .serializers import MyModelSerializer
    
    class MyModelCustomViewSet(mixins.ListModelMixin,
                                mixins.RetrieveModelMixin,
                                viewsets.GenericViewSet):
         queryset = MyModel.objects.all()
         serializer_class = MyModelSerializer
    ```
4. **ViewSet**: This is the most basic viewset that does not provide any default implementations. You need to define all the actions manually. It is useful when you want complete control over the behavior of your views.
    Example:
    ```python
    from rest_framework import viewsets
    from rest_framework.response import Response
    from .models import MyModel
    from .serializers import MyModelSerializer
    
    class MyModelViewSet(viewsets.ViewSet):
         def list(self, request):
              queryset = MyModel.objects.all()
              serializer = MyModelSerializer(queryset, many=True)
              return Response(serializer.data)
    
         def retrieve(self, request, pk=None):
              instance = MyModel.objects.get(pk=pk)
              serializer = MyModelSerializer(instance)
              return Response(serializer.data)
    ```
5. **ApiViewSet**: This type of viewset is used to create api endpoints. It is similar to ViewSet but is specifically designed for API views.
    Example:
    ```python
    from rest_framework import viewsets
    from rest_framework.response import Response
    from .models import MyModel
    from .serializers import MyModelSerializer
    
    class MyModelApiViewSet(viewsets.ApiViewSet):
        def list(self, request):
            queryset = MyModel.objects.all()
            serializer = MyModelSerializer(queryset, many=True)
            return Response(serializer.data)
    
        def retrieve(self, request, pk=None):
            instance = MyModel.objects.get(pk=pk)
            serializer = MyModelSerializer(instance)
            return Response(serializer.data)
    ```



## Viewset Parameters
Viewsets in Django REST Framework can be customized using various parameters. Here are some commonly used parameters:
1. **queryset**: This parameter defines the set of objects that the viewset will operate on. It is typically set to a Django QuerySet.

   Example:
   ```python
   queryset = MyModel.objects.all()
   ```
2. **serializer_class**: This parameter specifies the serializer class that will be used to serialize and deserialize data for the viewset.
    Example:
    ```python
    serializer_class = MyModelSerializer
    ```
3. **permission_classes**: This parameter defines the permission classes that will be used to control access to the viewset's actions.
    Example:
    ```python
    from rest_framework import permissions
    permission_classes = [permissions.IsAuthenticated]
    ```
4. **pagination_class**: This parameter specifies the pagination class that will be used to paginate the results returned by the viewset.
    Example:
    ```python
    from rest_framework.pagination import PageNumberPagination
    pagination_class = PageNumberPagination
    ```
5. **filter_backends**: This parameter defines the filter backends that will be used to filter the queryset based on request parameters.
    Example:
    ```python
    from rest_framework import filters
    filter_backends = [filters.SearchFilter, filters.OrderingFilter]
    ```
6. **search_fields**: This parameter specifies the fields that can be searched using the SearchFilter backend.
    Example:
    ```python
    search_fields = ['name', 'description']
    ```
7. **ordering_fields**: This parameter defines the fields that can be used for ordering the results using the OrderingFilter backend.
    Example:
    ```python
    ordering_fields = ['name', 'created_at']
    ```
8. **throttle_classes**: This parameter specifies the throttle classes that will be used to limit the rate of requests to the viewset.
    Example:
    ```python
    from rest_framework.throttling import UserRateThrottle
    throttle_classes = [UserRateThrottle]
    ```
9. **lookup_field**: This parameter defines the field that will be used to look up individual model instances. By default, it is set to 'pk'.
    Example:
    ```python
    lookup_field = 'slug'
    ```
10. **http_method_names**: This parameter specifies the allowed HTTP methods for the viewset.
    Example:
    ```python
    http_method_names = ['get', 'post', 'put', 'delete']
    ```
These parameters can be combined and customized to create viewsets that meet the specific needs of your application.

### Custom Actions
You can add custom endpoints using the `@action` decorator.
```python
from rest_framework.decorators import action
from rest_framework.response import Response

class UserViewSet(viewsets.ModelViewSet):
    ...

    # URL: /users/deactivate_all/ (POST)
    @action(detail=False, methods=['post'])
    def deactivate_all(self, request):
        self.get_queryset().update(is_active=False)
        return Response({'status': 'All users deactivated'})

    # URL: /users/{pk}/change_password/ (POST)
    @action(detail=True, methods=['post'])
    def change_password(self, request, pk=None):
        user = self.get_object()
        # logic here
        return Response({'status': 'password set'})
```

### Real World Example: Blog Post ViewSet
```python
class BlogPostViewSet(viewsets.ModelViewSet):
    queryset = BlogPost.objects.all()
    serializer_class = BlogPostSerializer
    permission_classes = [IsAuthenticatedOrReadOnly]
    filter_backends = [filters.SearchFilter, filters.OrderingFilter]
    search_fields = ['title', 'content']
    ordering_fields = ['created_at', 'title']

    def perform_create(self, serializer):
        serializer.save(author=self.request.user)
    
    @action(detail=True, methods=['post'])
    def publish(self, request, pk=None):
        post = self.get_object()
        post.is_published = True
        post.save()
        return Response({'status': 'published'})
```

### Viewset Methods
Viewsets in Django REST Framework provide several methods that correspond to standard CRUD operations along with other actions and utilities. Here are some commonly used viewset methods:
1. **list(self, request, *args, \*\*kwargs)**: This method is used to retrieve a list of objects. It corresponds to the HTTP GET method on the collection endpoint.
   Example:
   ```python
   def list(self, request, *args, **kwargs):
       queryset = self.get_queryset()
       serializer = self.get_serializer(queryset, many=True)
       return Response(serializer.data)
   ```
2. **retrieve(self, request, pk=None, *args, \*\*kwargs)**: This method is used to retrieve a single object by its primary key. It corresponds to the HTTP GET method on the detail endpoint.
   Example:
   ```python
   def retrieve(self, request, pk=None, *args, **kwargs):
       instance = self.get_object()
       serializer = self.get_serializer(instance)
       return Response(serializer.data)
   ```
3. **create(self, request, *args, \*\*kwargs)**: This method is used to create a new object. It corresponds to the HTTP POST method on the collection endpoint.
   Example:
   ```python
   def create(self, request, *args, **kwargs):
       serializer = self.get_serializer(data=request.data)
       serializer.is_valid(raise_exception=True)
       self.perform_create(serializer)
       return Response(serializer.data, status=status.HTTP_201_CREATED)
   ```
4. **update(self, request, pk=None, *args, \*\*kwargs)**: This method is used to update an existing object. It corresponds to the HTTP PUT method on the detail endpoint.
   Example:
   ```python
   def update(self, request, pk=None, *args, **kwargs):
       partial = kwargs.pop('partial', False)
       instance = self.get_object()
       serializer = self.get_serializer(instance, data=request.data, partial=partial)
       serializer.is_valid(raise_exception=True)
       self.perform_update(serializer)
       return Response(serializer.data)
   ```
5. **partial_update(self, request, pk=None, *args, \*\*kwargs)**: This method is used to partially update an existing object. It corresponds to the HTTP PATCH method on the detail endpoint.
   Example:
   ```python
   def partial_update(self, request, pk=None, *args, **kwargs):
       instance = self.get_object()
       serializer = self.get_serializer(instance, data=request.data, partial=True)
       serializer.is_valid(raise_exception=True)
       self.perform_update(serializer)
       return Response(serializer.data)
   ```
6. **destroy(self, request, pk=None, *args, \*\*kwargs)**: This method is used to delete an existing object. It corresponds to the HTTP DELETE method on the detail endpoint.
   Example:
   ```python
   def destroy(self, request, pk=None, *args, **kwargs):
       instance = self.get_object()
       self.perform_destroy(instance)
       return Response(status=status.HTTP_204_NO_CONTENT)
   ```  
7. **get_queryset(self)**: This method returns the queryset that will be used for the viewset's actions. It can be overridden to provide custom querysets based on request parameters.
   Example:
   ```python
   def get_queryset(self):
       return MyModel.objects.filter(owner=self.request.user)
   ```
8. **get_serializer(self, *args, \*\*kwargs)**: This method returns the serializer instance that will be used for the viewset's actions. It can be overridden to provide custom serializers based on request parameters.
   Example:
   ```python
   def get_serializer(self, *args, **kwargs):
       return MyModelSerializer(*args, **kwargs)
   ```
9. **get_serializer_class(self)**: This method returns the serializer class that will be used for the viewset's actions. It can be overridden to provide different serializers for different actions.
   Example:
   ```python
   def get_serializer_class(self):
       if self.action == 'list':
           return MyModelListSerializer
        elif self.action == 'retrieve':
            return MyModelDetailSerializer
       return MyModelDetailSerializer
   ```
10. **perform_create(self, serializer)**: This method is called when creating a new object. It can be overridden to add custom behavior during the creation process.
   Example:
   ```python
   def perform_create(self, serializer):
       serializer.save(owner=self.request.user)
   ```
11. **perform_update(self, serializer)**: This method is called when updating an existing object. It can be overridden to add custom behavior during the update process.
   Example:
   ```python
   def perform_update(self, serializer):
       serializer.save(updated_by=self.request.user)
   ```
12. **perform_destroy(self, instance)**: This method is called when deleting an existing object. It can be overridden to add custom behavior during the deletion process.
   Example:
   ```python
   def perform_destroy(self, instance):
       instance.delete()
   ```
13. **get_object(self)**: This method retrieves the object that the viewset is operating on. It can be overridden to provide custom object retrieval logic.
   Example:
   ```python
   def get_object(self):
       return MyModel.objects.get(slug=self.kwargs['slug'])
   ```
14. **paginate_queryset(self, queryset)**: This method paginates the given queryset if pagination is enabled. It can be overridden to provide custom pagination behavior.
   Example:
   ```python
   def paginate_queryset(self, queryset):
       return super().paginate_queryset(queryset)
   ```
15. **filter_queryset(self, queryset)**: This method filters the given queryset based on the filter backends. It can be overridden to provide custom filtering behavior.
    Example:
    ```python
    def filter_queryset(self, queryset):
         return super().filter_queryset(queryset)
    ```
16. **get_permissions(self)**: This method returns the list of permission classes that will be used for the viewset's actions. It can be overridden to provide different permissions for different actions.
    Example:
    ```python
    def get_permissions(self):
        if self.action == 'create':
            permission_classes = [permissions.IsAdminUser]
        else:
            permission_classes = [permissions.IsAuthenticated]
        return [permission() for permission in permission_classes]
    ```
17. **get_throttles(self)**: This method returns the list of throttle classes that will be used for the viewset's actions. It can be overridden to provide different throttles for different actions.
    Example:
    ```python
    def get_throttles(self):
        if self.action == 'list':
            throttle_classes = [UserRateThrottle]
        else:
            throttle_classes = [AnonRateThrottle]
        return [throttle() for throttle in throttle_classes]
    ``` 
18. **get_paginated_response(self, data)**: This method returns a paginated response for the given data. It can be overridden to provide custom paginated responses.
    Example:
    ```python
    def get_paginated_response(self, data):
        return super().get_paginated_response(data)
    ``` 
19. **initialize_request(self, request, *args, \*\*kwargs)**: This method initializes the request object for the viewset. It can be overridden to provide custom request initialization logic.
    Example:
    ```python
    def initialize_request(self, request, *args, **kwargs):
        return super().initialize_request(request, *args, **kwargs)
    ```
20. **dispatch(self, request, *args, \*\*kwargs)**: This method dispatches the request to the appropriate action method based on the HTTP method. It can be overridden to provide custom dispatch logic.
    Example:
    ```python
    def dispatch(self, request, *args, **kwargs):
        return super().dispatch(request, *args, **kwargs)
    ```
21. **handle_exception(self, exc)**: This method handles exceptions that occur during the processing of a request. It can be overridden to provide custom exception handling logic.
    Example:
    ```python
    def handle_exception(self, exc):
        return super().handle_exception(exc)
    ```
22. **options(self, request, *args, \*\*kwargs)**: This method handles HTTP OPTIONS requests. It can be overridden to provide custom behavior for OPTIONS requests.
    Example:
    ```python
    def options(self, request, *args, **kwargs):
        return super().options(request, *args, **kwargs)
    ``` 
23. **head(self, request, *args, \*\*kwargs)**: This method handles HTTP HEAD requests. It can be overridden to provide custom behavior for HEAD requests.
    Example:
    ```python
    def head(self, request, *args, **kwargs):
        return super().head(request, *args, **kwargs)
    ```
24. **finalize_response(self, request, response, *args, \*\*kwargs)**: This method finalizes the response object before it is returned to the client. It can be overridden to provide custom response finalization logic.
    Example:
    ```python
    def finalize_response(self, request, response, *args, **kwargs):
        return super().finalize_response(request, response, *args, **kwargs)
    ```